In [222]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [223]:
#Read in the csv
lending_data = pd.read_csv("LoanStats_2019Q1.csv")

In [224]:
lending_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,60 months,17.19%,499.10,C,C5,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,12000.0,12000.0,12000.0,36 months,16.40%,424.26,C,C4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3000.0,3000.0,3000.0,36 months,14.74%,103.62,C,C2,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,35000.0,35000.0,35000.0,36 months,15.57%,1223.08,C,C3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,5000.0,5000.0,5000.0,36 months,15.57%,174.73,C,C3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
lending_df = lending_data[["loan_status","grade","sub_grade","annual_inc","dti","addr_state","int_rate"]]
status = lending_df.groupby("loan_status").count()
status

,grade,sub_grade,annual_inc,dti,addr_state,int_rate
loan_status,,,,,,
Charged Off,176,176,176,176,176,176
Current,109176,109176,109176,108936,109176,109176
Fully Paid,4730,4730,4730,4716,4730,4730
In Grace Period,538,538,538,537,538,538
Late (16-30 days),260,260,260,259,260,260
Late (31-120 days),795,795,795,794,795,795


In [226]:
#Create the upsample and downsample
small_sample = lending_df[(lending_df["loan_status"] == "Charged Off") | (lending_df["loan_status"] == "Late (31-120 days)")]
large_sample = lending_df[(lending_df["loan_status"] == "Fully Paid")]


In [227]:
small_sample.head()
large_sample.head()

,loan_status,grade,sub_grade,annual_inc,dti,addr_state,int_rate
64,Fully Paid,C,C3,65000.0,15.18,CO,15.57%
87,Fully Paid,C,C5,82000.0,15.70,MO,17.19%
91,Fully Paid,C,C1,90000.0,7.92,NY,13.90%
105,Fully Paid,D,D5,17808.0,30.12,OH,28.80%
133,Fully Paid,B,B4,38000.0,38.66,GA,12.40%


In [228]:
n_class0 = len(small_sample)
n_class1 = len(large_sample)

under = large_sample.sample(n_class0)
sample_df = pd.concat([under, small_sample], axis=0)

len(sample_df)


1942

In [241]:
sample_df["loan_category"] = np.where(sample_df["loan_status"] == "Fully Paid", 'good', 'risky')
sample_df.head()

,loan_status,grade,sub_grade,annual_inc,dti,addr_state,int_rate,loan_category
53135,Fully Paid,A,A2,95000.0,9.84,NM,7.02%,good
31737,Fully Paid,D,D2,108202.0,18.69,MI,20.00%,good
60717,Fully Paid,A,A3,87170.0,23.08,GA,7.56%,good
32905,Fully Paid,C,C1,140000.0,8.49,NY,13.90%,good
85203,Fully Paid,D,D1,75000.0,8.51,CA,17.97%,good


In [258]:
sample_df.sort_values(by="int_rate", ascending=True)

,loan_status,grade,sub_grade,annual_inc,dti,addr_state,int_rate,loan_category
87339,Fully Paid,B,B1,60000.0,9.06,TX,10.33%,good
98202,Fully Paid,B,B1,70000.0,12.05,IN,10.33%,good
37286,Charged Off,B,B1,235000.0,0.28,IL,10.33%,risky
38462,Late (31-120 days),B,B1,60000.0,3.56,WA,10.33%,risky
41370,Late (31-120 days),B,B1,72800.0,26.47,VA,10.33%,risky
42055,Charged Off,B,B1,227600.0,9.07,IL,10.33%,risky
111885,Fully Paid,B,B1,165000.0,5.04,TX,10.33%,good
112478,Fully Paid,B,B1,105000.0,16.20,OH,10.33%,good
43622,Charged Off,B,B1,70000.0,27.12,IN,10.33%,risky
44698,Charged Off,B,B1,43000.0,16.94,IL,10.33%,risky


In [259]:
X = sample_df[["grade", "sub_grade","annual_inc", "dti", "int_rate"]]
y = sample_df["loan_category"].values.reshape(-1,1)
print(X.shape, y.shape)

(1942, 5) (1942, 1)


In [260]:
#making loan_status binary by using loan category

data_binary_encoded = pd.get_dummies(sample_df, columns=["loan_category"])
pd.DataFrame(data_binary_encoded).head()


,loan_status,grade,sub_grade,annual_inc,dti,addr_state,int_rate,loan_category_good,loan_category_risky
53135,Fully Paid,A,A2,95000.0,9.84,NM,7.02%,1,0
31737,Fully Paid,D,D2,108202.0,18.69,MI,20.00%,1,0
60717,Fully Paid,A,A3,87170.0,23.08,GA,7.56%,1,0
32905,Fully Paid,C,C1,140000.0,8.49,NY,13.90%,1,0
85203,Fully Paid,D,D1,75000.0,8.51,CA,17.97%,1,0


In [261]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data)
data_binary_encoded.head()

,annual_inc,dti,grade_A,grade_B,grade_C,grade_D,grade_E,grade_G,sub_grade_A1,sub_grade_A2,...,int_rate_28.90%,int_rate_28.97%,int_rate_29.00%,int_rate_30.79%,int_rate_30.84%,int_rate_6.46%,int_rate_7.02%,int_rate_7.56%,int_rate_8.19%,int_rate_8.81%
53135,95000.0,9.84,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
31737,108202.0,18.69,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60717,87170.0,23.08,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32905,140000.0,8.49,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85203,75000.0,8.51,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [262]:
#Scaling and normalization
#!) Train/test/split
from sklearn.model_selection import train_test_split

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()


,annual_inc,dti,grade_A,grade_B,grade_C,grade_D,grade_E,grade_G,sub_grade_A1,sub_grade_A2,...,int_rate_28.90%,int_rate_28.97%,int_rate_29.00%,int_rate_30.79%,int_rate_30.84%,int_rate_6.46%,int_rate_7.02%,int_rate_7.56%,int_rate_8.19%,int_rate_8.81%
102909,250001.0,9.78,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86052,47000.0,11.21,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
107082,92000.0,17.86,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45963,45000.0,20.37,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96904,82000.0,21.51,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [263]:
#Scale the data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

/Users/kenmurakami/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


ValueError: could not convert string to float: 'good'